In [18]:
import numpy as np
import xarray as xr
import csv
import datetime
import pandas as pd
from pyproj import Transformer
import glob

In [19]:
## Pandas datetime to decimal year
def year_fraction(date):
    start = datetime.date(date.year, 1, 1).toordinal()
    year_length = datetime.date(date.year+1, 1, 1).toordinal() - start
    return date.year + float(date.toordinal() - start) / year_length

In [23]:
## FDMs in the order of IMAU, GSFC, and MAR
IMAU2020 = "/Volumes/Samsung_T5/Greenland_climatology/RACMO/IMAUFDM_RACMO2.3p2_1960-2020/FDM_zs_FGRN055_1957-2020_GrIS_GIC.nc"
GSFC_FDM = "/Volumes/Samsung_T5/Greenland_climatology/MERRA_2/gsfc_fdm_v1.2/gsfc_fdm_v1_2_1_gris_June22.nc"
MAR_FDM = "/Volumes/Samsung_T5/Greenland_climatology/MAR/MARv3.12.1-6.5km-daily-ERA5-1979/"

f_names = [IMAU2020, GSFC_FDM, MAR_FDM]
h_names = ['zs', 'h_a', 'ZN6']
output_dir = ""
outputf_names = ["FDM_zs_FGRN055_1957-2020_GrIS_GIC_SERACInput.csv", "gsfc_fdm_v1_2_1_gris_June22_SERACInput.csv", "MARv3.12.1-6.5km-daily-ERA5_SERACInput.csv"]

##------------------------------------Change f_source and l_ind------------------------------------
f_source = "MAR" # IMAU, GSFC, or MAR
l_ind = 2 # 0 (IMAU), 1 (GSFC), 2 (MAR)

f_name = f_names[l_ind]
h_name = h_names[l_ind]
outputf_name = outputf_names[l_ind]

In [24]:
%%time

## Read FDM time and data
if f_source == "MAR":
    f_list = glob.glob(MAR_FDM+"*")

    def sortfunc(n):
        return n[-7:-3]
    f_list.sort(key=sortfunc)


    FDM = xr.open_dataset(f_list[0])
    data = FDM[h_name].values
    time = FDM['TIME'].values

    for f in f_list[1:]:
        FDM = xr.open_dataset(f)
        data = np.concatenate((data, FDM['ZN6'].values), axis=0)
        time = np.concatenate((time, FDM['TIME'].values))
    data = np.squeeze(data)
    data = np.swapaxes(data, 0, 2)

elif f_source == "IMAU" or f_source == "GSFC":
    FDM = xr.open_dataset(f_name)
    time = FDM['time'].values
    data = FDM[h_name].values
    data = np.swapaxes(data, 0, 2)
    data = np.swapaxes(data, 0, 1)

    
time_FDM = []
if type(time[0]) == np.datetime64:
    for x in time:
        time_FDM.append(year_fraction(pd.Timestamp(x).to_pydatetime()))
else:
    time_FDM = time

CPU times: user 1min 45s, sys: 4min 44s, total: 6min 30s
Wall time: 11min 44s


In [25]:
%%time

## Prepare data
data = data.reshape(-1, len(time_FDM))
if f_source == "MAR":
    data = np.cumsum(data, axis = 1)

## Prepare header
header = ['Lon','Lat','X','Y']
header.extend(np.round(time_FDM, 4))



if f_source == "IMAU":
    # Coordinates: Lat, Lon (epsg4326), X, Y (epsg32624)
    transformer = Transformer.from_crs("epsg:4326", "epsg:32624", always_xy=True)
    lon = FDM.lon.values.reshape(-1)
    lat = FDM.lat.values.reshape(-1)
    X, Y = pd.Series(transformer.transform(lon, lat))
elif f_source == "GSFC" or f_source == "MAR":
    # Coordinates: Lat, Lon (epsg4326), X, Y (epsg32624)
    x2d, y2d = np.meshgrid(FDM['x'].values, FDM['y'].values)

    transformer = Transformer.from_crs("epsg:3413", "epsg:32624", always_xy=True)
    X, Y = pd.Series(transformer.transform(x2d.flatten(), y2d.flatten()))

    transformer = Transformer.from_crs("epsg:3413", "epsg:4326", always_xy=True)
    lon, lat = pd.Series(transformer.transform(x2d.flatten(), y2d.flatten()))

    
coord = np.array([lon, lat, np.round(X), np.round(Y)]).T

if f_source == "MAR":
    msk = ~(data == 0.).all(axis=1)
else:
    msk = ~np.isnan(data).any(axis=1)

data = np.concatenate([coord[msk], data[msk]], axis=1)
data = np.round(data, 6)

CPU times: user 5min 58s, sys: 30min 17s, total: 36min 15s
Wall time: 1h 1min 13s


In [26]:
%%time

with open(output_dir+outputf_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f, delimiter=',')

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

CPU times: user 8min 33s, sys: 24.3 s, total: 8min 57s
Wall time: 9min 59s
